In [64]:
import plotly.express as px
import math

In [65]:
class Order():
    def __init__(self,price,volume,isBuy):
        self.price = price
        self.volume = volume
        self.isBuy = isBuy
        self.unfilledVolume = volume
        self.executionPrices = []

    def __str__(self):
        return f"Price={self.price}, Volume={self.volume}, Unfilled={self.unfilledVolume}"

    def execute(self,execution): #in the future can change this to log when orders are filled or execute something or whatever (amount,price)
        self.executionPrices.append(execution)
        self.unfilledVolume -= execution[0]



In [66]:
class Node():
    def __init__(self,value,orders = None):
        self.value = value
        self.left = None
        self.right = None
        self.parent = None
        if orders == None: #for increased efficiency change orders to a linked list but I am to lazy to do that right now
            self.orders = []
        else:
            self.orders = orders

    def __str__(self): #does inorder traversal returning all of the orders
        output  = []
        for order in self.orders:
            output.append(str(order))
        return ".\n".join(output)

    def in_order(self):
        if self.left != None:
            yield from self.left.in_order()

        yield self.orders,self
        
        if self.right != None:
            yield from self.right.in_order()


    def insert(self,order,direction): 
        if order.price == self.value:
            self.orders.append(order) 

        elif order.price * direction < self.value * direction: #move to right node
            if self.right == None:
                self.right = Node(order.price,[order])
                self.right.parent = self
            else:
                self.right.insert(order,direction)

        else: #move to left node
            if self.left == None:
                self.left = Node(order.price,[order])
                self.left.parent = self
            else:
                self.left.insert(order,direction)



In [67]:
class OrderTree():
    def __init__(self,isBuy,node = None):
        self.head = node
        self.isBuy = isBuy

    def bestPrice(self):
        if self.head == None:
            if self.isBuy:
                return -math.inf
            else:
                return math.inf
        else:
            currentNode = self.head
            while currentNode.left is not None:
                currentNode = currentNode.left
            return currentNode.value

    def __str__(self):
        header = f"bestPrice={self.bestPrice()}"
        if self.head == None:
            tree = "empty"
        else:
            tree = ".\n".join([str(order) for orders,_ in self.head.in_order() for order in orders])
        return header + '\n' + tree


    def insertOrder(self,order): 
        if self.head is None: #to optimize use a self balancing tree
            self.head = Node(order.price, [order])
        else:
            direction = 1 if self.isBuy else -1
            self.head.insert(order, direction)
        '''if self.isBuy:

            if order.price > self.bestPrice(): 
                newNode = Node(order.price,[order])
                if self.head == None:
                    self.head = newNode
                else:
                    newNode.right = self.head
                    self.head = newNode
                    self.head.right.parent = newNode
            else:
                self.head.insert(order,1)

        else:

            if order.price < self.bestPrice(): 
                newNode = Node(order.price,[order])
                if self.head == None:
                    self.head = newNode
                else:
                    newNode.right = self.head
                    self.head = newNode
                    self.head.right.parent = newNode
            else:
                self.head.insert(order,-1)'''

In [68]:
class orderbook:
    def __init__(self):
        self.sells = OrderTree(False)
        self.buys = OrderTree(True)


    def __str__(self):
        return "---------------\nBuy Book \n---------------\n" + str(self.buys) + "\n" + "---------------\nSell Book \n---------------\n" + str(self.sells) 

    def submitOrder(self,order): #ensure that it checks that the price is still currently a valid price

        deleteNodes = []
        
        if order.isBuy: #submitting a buy order
            if order.price < self.sells.bestPrice(): #no order fill
                self.buys.insertOrder(order)
            else: #order fillable
                for existingOrders, currentNode in self.sells.head.in_order(): # fills order on root node

                    if order.price < currentNode.value:
                        break
                    updatedOrders = []

                    for existingOrder in existingOrders:
                        if existingOrder.unfilledVolume <= order.unfilledVolume:
                            #fill entire order
                            existingOrder.execute((existingOrder.unfilledVolume,existingOrder.price))
                            order.execute((existingOrder.unfilledVolume,existingOrder.price))
                        else:
                            #partial fill the order
                            existingOrder.execute((order.unfilledVolume,existingOrder.price)) #adds tuple of trade execution to the order and sets remaning volume to zero
                            order.execute((order.unfilledVolume,existingOrder.price))
                            updatedOrders.append(existingOrder)
                            break
                        if existingOrder.unfilledVolume > 0:
                            updatedOrders.append(existingOrder)

                    currentNode.orders = updatedOrders
                    if len(updatedOrders) == 0:
                        deleteNodes.append(currentNode)


                    if order.unfilledVolume <= 0:#order is completely filled
                        break

                if order.unfilledVolume > 0:#insert remaining unfilled volume onto the market
                    self.buys.insertOrder(order)

                for node in deleteNodes:
                    if node.parent == None: 
                        self.sells.head = self.sells.head.right
                        if self.sells.head != None:
                            self.sells.head.parent = None
                    else:
                        if node.right == None:
                            node.parent.left = node.right
                        else:
                            node.right.parent = node.parent
                            node.parent.left = node.right
                        



        else: #submitting a sell order
            if order.price > self.buys.bestPrice(): #no order fill
                self.sells.insertOrder(order)
            else: #order fillable
                for existingOrders, currentNode in self.buys.head.in_order(): # fills order on root node

                    if order.price > currentNode.value:
                        break
                    updatedOrders = []

                    for existingOrder in existingOrders:
                        if existingOrder.unfilledVolume <= order.unfilledVolume:
                            #fill entire order
                            existingOrder.execute((existingOrder.unfilledVolume,existingOrder.price))
                            order.execute((existingOrder.unfilledVolume,existingOrder.price))
                        else:
                            #partial fill the order
                            existingOrder.execute((order.unfilledVolume,existingOrder.price)) #adds tuple of trade execution to the order and sets remaning volume to zero
                            order.execute((order.unfilledVolume,existingOrder.price))
                            updatedOrders.append(existingOrder)
                            break
                        if existingOrder.unfilledVolume > 0:
                            updatedOrders.append(existingOrder)

                    currentNode.orders = updatedOrders
                    if len(updatedOrders) == 0:
                        deleteNodes.append(currentNode)


                    if order.unfilledVolume <= 0:#order is completely filled
                        break

                if order.unfilledVolume > 0: #insert remaining unfilled volume onto the market
                    self.sells.insertOrder(order)
        
                for node in deleteNodes:
                    if node.parent == None: 
                        self.buys.head = self.buys.head.right
                        if self.buys.head != None:
                            self.buys.head.parent = None
                    else:
                        if node.right == None:
                            node.parent.left = node.right
                        else:
                            node.right.parent = node.parent
                            node.parent.left = node.right

In [69]:
exchange = orderbook()

order1 = Order(100,40,True)
order2 = Order(120,40,True)


exchange.submitOrder(order1)
exchange.submitOrder(order2)

a = Order(150,40,False)
exchange.submitOrder(a)


a = Order(100,60,False)
exchange.submitOrder(a)

print(exchange)


---------------
Buy Book 
---------------
bestPrice=-inf
empty
---------------
Sell Book 
---------------
bestPrice=100
Price=100, Volume=60, Unfilled=60.
Price=150, Volume=40, Unfilled=40


In [107]:
import random
exchange = orderbook()

for x in range(10):
    order = Order(random.randint(100, 200),100,False)
    exchange.submitOrder(order)
    order2 = Order(random.randint(100, 200),100,True)
    exchange.submitOrder(order2)


print(exchange)

---------------
Buy Book 
---------------
bestPrice=130
Price=130, Volume=100, Unfilled=100.
Price=100, Volume=100, Unfilled=100
---------------
Sell Book 
---------------
bestPrice=149
Price=149, Volume=100, Unfilled=100.
Price=180, Volume=100, Unfilled=100.
Price=188, Volume=100, Unfilled=100.
Price=196, Volume=100, Unfilled=100
